**Overview**

*   In LangChain, **tools** are a way to associate a Python function with a schema.
*   This schema defines the function's **name**, **description**, and **expected arguments**.
*   Tools can be used by chat models that support tool calling, allowing the model to request the execution of a function with specific inputs.
*   Tools are designed so that their outputs can be fed back into the model.

**Key Concepts**

*   Tools encapsulate a function and its schema, allowing them to be passed to chat models.
*   The `@tool` decorator is the recommended way to create tools.
    *   It automatically infers the tool's **name**, **description**, and **expected arguments**.
    *   It supports customisation.
    *   It allows for defining tools that return **artifacts** (e.g., images, dataframes).
    *   It allows for hiding input arguments from the schema using **injected tool arguments**.

**Tool Interface**

*   The tool interface is defined in the `BaseTool` class, a subclass of the `Runnable` interface.
*   Key attributes that correspond to the tool's **schema** include:
    *   **`name`**: The name of the tool.
    *   **`description`**: A description of what the tool does.
    *   **`args`**: A property that returns the JSON schema for the tool's arguments.
*   Key methods for executing the function associated with the **tool** include:
    *   **`invoke`**: Invokes the tool with the given arguments.
    *   **`ainvoke`**: Invokes the tool with the given arguments, asynchronously.

**Creating Tools**

*   The `@tool` decorator simplifies tool creation and is the recommended method.
*   After defining a function, decorating it with `@tool` creates a tool that implements the Tool Interface.
*   Example using the `@tool` decorator:
    ```python
    from langchain_core.tools import tool

    @tool
    def multiply(a: int, b: int) -> int:
        """Multiply two numbers."""
        return a * b
    ```
*   Other ways to create tools exist, such as subclassing `BaseTool` or using `StructuredTool`, but the `@tool` decorator is generally preferred.

**Using Tools Directly**

*   Tools can be used directly by calling the function with arguments.
*   Example:
    ```python
    multiply.invoke({"a": 2, "b": 3})
    ```
*   The tool's schema and other properties can be inspected:
    ```python
    print(multiply.name)   # Output: multiply
    print(multiply.description)  # Output: Multiply two numbers.
    print(multiply.args)  # Output: {'type': 'object', 'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b']}
    ```
*   Direct interaction with tools may not be necessary when using pre-built LangChain or LangGraph components, but it can be valuable for debugging, testing and building custom workflows.

**Configuring the Schema**

*   The `@tool` decorator offers options to modify the schema (e.g., name, description) or parse the function's docstring to infer the schema.

**Tool Artifacts**

*   Tools can return **artifacts** that can be accessed by downstream components but not exposed to the model directly.
*   This is useful for things like custom objects, dataframes, or images.
*   The `@tool` decorator can be used with `response_format="content_and_artifact"` to return a tuple of a message for the chat model and an artifact.

**Special Type Annotations**

*   Special type annotations can be used in a tool's function signature to configure runtime behaviour.
*   The following type annotations will **remove** the argument from the tool's schema:
    *   **`InjectedToolArg`**: Value is injected manually at runtime using `.invoke` or `.ainvoke`.
    *   **`RunnableConfig`**: Passes the `RunnableConfig` object to the tool.
    *  **`InjectedState`**: Passes the overall state of the LangGraph graph to the tool.
    *   **`InjectedStore`**: Passes the LangGraph store object to the tool.
*   The `Annotated` type with a string literal adds a **description** to the corresponding argument, which will be exposed in the schema.
*   `InjectedToolArg` allows parameters to be hidden from the tool's schema and to be passed at runtime. For example, to inject a `user_id` at runtime:
    ```python
    from langchain_core.tools import tool, InjectedToolArg

    @tool
    def user_specific_tool(input_data: str, user_id: InjectedToolArg) -> str:
        """Tool that processes input data."""
        return f"User {user_id} processed {input_data}"
    ```
*   The `RunnableConfig` object can be accessed within a tool by using the `RunnableConfig` annotation. This is injected at runtime and is not part of the tool's schema.
    ```python
    from langchain_core.runnables import RunnableConfig

    @tool
    async def some_func(..., config: RunnableConfig) -> ...:
        """Tool that does something."""
        # do something with config
        ...
        await some_func.ainvoke(..., config={"configurable": {"value": "some_value"}})
    ```

**Best Practices**

*   Tools should be well-named, well-documented, and properly type-hinted.
*   Tools should be simple and narrowly scoped.
*   Chat models with tool-calling APIs should be used to take advantage of tools.

**Toolkits**

*   Toolkits are a thin abstraction that group tools designed to be used together for specific tasks.
*   Toolkits expose a `get_tools` method that returns a list of tools.

**Related Resources**

*   API Reference for `@tool`
*   How to create custom tools
*   How to pass runtime values to tools
*   All LangChain tool how-to guides
*   Additional how-to guides that show usage with LangGraph
*   Tool integrations documentation.


In [1]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
   """Multiply two numbers."""
   return a * b

In [2]:
multiply.invoke({"a": 2, "b": 3})

6

In [3]:
print(multiply.name) # multiply
print(multiply.description) # Multiply two numbers.
print(multiply.args) 

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [11]:
from langchain_core.tools import tool, InjectedToolArg


# InjectedToolArg should be inserted separately https://python.langchain.com/docs/how_to/tool_runtime/

@tool
def user_specific_tool(input_data: str, user_id: InjectedToolArg) -> str:
    """Tool that processes input data."""
    return f"User {user_id} processed {input_data}"

In [ ]:
from langchain_core.runnables import RunnableConfig

@tool
async def some_func(..., config: RunnableConfig) -> ...:
    """Tool that does something."""
    # do something with config
    ...

await some_func.ainvoke(..., config={"configurable": {"value": "some_value"}})

In [12]:
from langchain_ollama import ChatOllama
model = ChatOllama(model="llama3.1", verbose=True)

In [ ]:
??tool

Signature:
tool(
    name_or_callable: Union[str, Callable, NoneType] = None,
    runnable: Optional[langchain_core.runnables.base.Runnable] = None,
    *args: Any,
    return_direct: bool = False,
    args_schema: Optional[type] = None,
    infer_schema: bool = True,
    response_format: Literal['content', 'content_and_artifact'] = 'content',
    parse_docstring: bool = False,
    error_on_invalid_docstring: bool = True,
) -> Union[langchain_core.tools.base.BaseTool, Callable[[Union[Callable, langchain_core.runnables.base.Runnable]], langchain_core.tools.base.BaseTool]]
Source:   
def tool(
    name_or_callable: Optional[Union[str, Callable]] = None,
    runnable: Optional[Runnable] = None,
    *args: Any,
    return_direct: bool = False,
    args_schema: Optional[type] = None,
    infer_schema: bool = True,
    response_format: Literal["content", "content_and_artifact"] = "content",
    parse_docstring: bool = False,
    error_on_invalid_docstring: bool = True,
) -> Union[
    BaseTo